In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mord as m

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, Imputer, Normalizer, StandardScaler

# sklearn meteics for model performance
from sklearn.metrics import confusion_matrix,roc_curve, auc, roc_auc_score

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# for splitting data into train and test
from sklearn.cross_validation import train_test_split

import logging
logging.basicConfig(level=logging.DEBUG)

import statsmodels.api as sm
from patsy.highlevel import dmatrices

%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


# Read Data

In [2]:
build_owner = pd.read_csv("Dataset/Building_Ownership_Use.csv")
build_struct = pd.read_csv("Dataset/Building_Structure.csv")
train = pd.read_csv("Dataset/train.csv")
test = pd.read_csv("Dataset/test.csv")


#### Details of the files are as follows: 

#### train.csv : 

|Varaible| Descrition|
|--------|-----------|
|area_assesed|Indicates the nature of the damage assessment in terms of the areas of the building that were assessed|
|building_id|A unique ID that identifies every individual building|
|damage_grade|Damage grade assigned to the building after assessment (Target Variable)|
|district_id|District where the building is located|
|has_geotechnical_risk|Indicates if building has geotechnical risks|
|has_geotechnical_risk_fault_crack|Indicates if building has geotechnical risks related to fault cracking|
|has_geotechnical_risk_flood|Indicates if building has geotechnical risks related to flood|
|has_geotechnical_risk_land_settlement|Indicates if building has geotechnical risks related to land settlement|
|has_geotechnical_risk_landslide|Indicates if building has geotechnical risks related to landslide|
|has_geotechnical_risk_liquefaction|Indicates if building has geotechnical risks related to liquefaction|
|has_geotechnical_risk_other|Indicates if building has any other  geotechnical risks|
|has_geotechnical_risk_rock_fall|Indicates if building has geotechnical risks related to rock fall|
|has_repair_started|Indicates if the repair work had started|
|vdcmun_id|Municipality where the building is located|

#### test.csv

Contains the same variables as the train.csv except the 'damage_grade' which is the target variable/ varaible to be predicted.

#### Building_Ownership_Use.csv: 

|Varaible|Description|
|--------|-----------|
|building_id|A unique ID that identifies every individual building|
|district_id|District where the building is located|
|vdcmun_id|Municipality where the building is located|
|ward_id|Ward Number in which the building is located|
|legal_ownership_status|Legal ownership status of the land in which the building was built|
|count_families|Number of families in the building|
|has_secondary_use|indicates if the building is used for any secondary purpose|
|has_secondary_use_agriculture|indicates if the building is secondarily used for agricultural purpose|
|has_secondary_use_hotel|indicates if the building is secondarily used as hotel|
|has_secondary_use_rental|indicates if the building is secondarily used for rental purpose|
|has_secondary_use_institution|indicates if the building is secondarily used for institutional purpose|
|has_secondary_use_school|indicates if the building is secondarily used as school|
|has_secondary_use_industry|indicates if the building is secondarily used for industrial purpose|
|has_secondary_use_health_post|indicates if the building is secondarily used as health post|
|has_secondary_use_gov_office|indicates if the building is secondarily used as government office|
|has_secondary_use_use_police|indicates if the building is secondarily used as police station|
|has_secondary_use_other|indicates if the building is secondarily used for other purpose|


#### Building_Structure.csv

|Variable|Description|
|--------|-----------|
|building_id|A unique ID that identifies every individual building|
|district_id|District where the building is located|
|vdcmun_id|Municipality where the building is located|
|ward_id|Ward Number in which the building is located|
|count_floors_pre_eq|Number of floors that the building had before the earthquake|
|count_floors_post_eq|Number of floors that the building had after the earthquake|
|age_building|Age of the building (in years)|
|plinth_area_sq_ft|Plinth area of the building (in square feet)|
|height_ft_pre_eq|Height of the building before the earthquake (in feet)|
|height_ft_post_eq|Height of the building after the earthquake (in feet)|
|land_surface_condition|Surface condition of the land in which the building is built	|
|foundation_type|Type of foundation used in the building|
|roof_type|Type of roof used in the building|
|ground_floor_type|Ground floor type|
|other_floor_type|Type of construction used in other floors (except ground floor and roof)|
|position|Position of the building|
|plan_configuration|Building plan configuration|
|has_superstructure_adobe_mud|indicates if the superstructure of the building is made of Adobe/Mud|
|has_superstructure_mud_mortar_stone|indicates if the superstructure of the building is made of Mud Mortar - Stone|
|has_superstructure_stone_flag| indicates if the superstructure of the building is made of Stone|
|has_superstructure_mud_mortar_brick|indicates if the superstructure of the building is made of Cement Mortar - Stone|
|has_superstructure_cement_mortar_brick|indicates if the superstructure of the building is made of Mud Mortar - Brick|
|has_superstructure_timber|indicates if the superstructure of the building is made of Timber|
|has_superstructure_bamboo|indicates if the superstructure of the building is made of Bamboo|
|has_superstructure_rc_non_engineered|indicates if the superstructure of the building is made of RC (Non Engineered)|
|has_superstructure_rc_engineered|indicates if the superstructure of the building is made of RC (Engineered)|
|has_superstructure_other| indicates if the superstructure of the building is made of any other material|
|condition_post_eq|Actual contition of the building after the earthquake|


In [3]:
len(train) + len(test), len(build_owner), len(build_struct)

(1052936, 1052948, 1052948)

In [4]:
data = (train.merge(build_struct, how='inner', on=['building_id','district_id','vdcmun_id']))
data = (data.merge(build_owner, how='inner', on=['building_id','district_id','vdcmun_id','ward_id']))

combined_set = pd.concat([train.loc[:,train.columns!='damage_grade'], test], axis = 0) # Stacks them vertically
combined_set = (combined_set.merge(build_struct, how='inner', on=['building_id','district_id','vdcmun_id']))
combined_set = (combined_set.merge(build_owner, how='inner', on=['building_id','district_id','vdcmun_id','ward_id']))

In [5]:
data.head()

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Exploratory Data Analysis

## Column Values


Let's look at the number of columns of each data type. **int64** and **float64** are numeric variables (which can be either discrete or continuous). **object** columns contain strings and are categorical features.

In [6]:
print(f'dtype value counts:\n{data.dtypes.value_counts()}')
data_types = pd.concat([data.dtypes, data.apply(pd.Series.nunique)], axis=1)
data_types = data_types.rename(columns = {0: "dtype", 1: "#unique values"})
data_types.sort_values(['#unique values'])

dtype value counts:
int64      37
object     12
float64     4
dtype: int64


,dtype,#unique values
has_secondary_use_other,int64,2
has_superstructure_cement_mortar_stone,int64,2
has_superstructure_stone_flag,int64,2
has_superstructure_mud_mortar_stone,int64,2
has_superstructure_adobe_mud,int64,2
has_superstructure_timber,int64,2
has_secondary_use_use_police,int64,2
has_superstructure_bamboo,int64,2
has_superstructure_rc_non_engineered,int64,2
has_superstructure_rc_engineered,int64,2


## Change data types to correct data types

In [7]:
combined_set['ward_id'] = combined_set['ward_id'].astype(object)
combined_set['vdcmun_id'] = combined_set['vdcmun_id'].astype(object)
combined_set['district_id'] = combined_set['district_id'].astype(object)

## Examine Missing Values

In [8]:
def missing_values_table(data):
    # Total missing values
    mis_val = data.isnull().sum()

    # Percent of missing values
    mis_val_percent = 100 * data.isnull().sum()/len(data)

    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis = 1)

    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: "Missing Values", 1: "% of Total Values"})

    # Concatenate the datatype information
    data_types = pd.concat([data.dtypes, data.apply(pd.Series.nunique)], axis=1)
    data_types = data_types.rename(columns = {0: "dtype", 1: "#unique values"})
    mis_val_table_ren_columns = pd.concat([data_types, mis_val_table_ren_columns], axis=1)
    
    # Sort the table by percent of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values("% of Total Values", ascending = False).round(1)
         
    
    # Print some summary information
    print ("Your selected dataframe has " + str(data.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,3] != 0].shape[0]) +
            " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [9]:
# Missing values statistics
missing_values = missing_values_table(combined_set)
#missing_values

Your selected dataframe has 52 columns.
There are 1 columns that have missing values.


## Examine the distribution of target column

In [10]:
# Fraction of signup drivers who completed the first trip
target_dist = pd.concat([data['damage_grade'].value_counts(), 100 * data['damage_grade'].value_counts()/len(data)], axis=1)
target_dist.columns = ['target_count','target_fraction (in %)']
target_dist

,target_count,target_fraction (in %)
Grade 5,210825,33.371006
Grade 4,152244,24.098354
Grade 3,122288,19.356687
Grade 2,85084,13.467751
Grade 1,61320,9.706202


# Feature Engineering

In [11]:
combined_set['change_floors_post_eq'] = combined_set['count_floors_post_eq'] - combined_set['count_floors_pre_eq']

In [12]:
combined_set['change_height_ft_post_eq'] = combined_set['height_ft_post_eq'] - combined_set['height_ft_pre_eq']

In [13]:
combined_set = combined_set.drop(columns=['building_id','ward_id','vdcmun_id'])

In [14]:
missing_values_table(combined_set.loc[:,combined_set.dtypes=='object'])

Your selected dataframe has 11 columns.
There are 0 columns that have missing values.


,dtype,#unique values,Missing Values,% of Total Values
area_assesed,object,5,0,0.0
district_id,object,31,0,0.0
land_surface_condition,object,3,0,0.0
foundation_type,object,5,0,0.0
roof_type,object,3,0,0.0
ground_floor_type,object,5,0,0.0
other_floor_type,object,4,0,0.0
position,object,4,0,0.0
plan_configuration,object,10,0,0.0
condition_post_eq,object,8,0,0.0


# Imputing Missing Values

In [15]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [16]:
combined_set = DataFrameImputer().fit_transform(combined_set)
#missing_values_table(combined_set)

# Encoding Categorical Variables

In [17]:
data['damage_grade'].replace({'Grade 4':4, 'Grade 2':2, 'Grade 1':1, 'Grade 5':5, 'Grade 3':3},inplace=True)

In [18]:
combined_set['district_id'] = combined_set['district_id'].astype(object)
missing_values_table(combined_set.loc[:,combined_set.dtypes == 'object'])

Your selected dataframe has 11 columns.
There are 0 columns that have missing values.


,dtype,#unique values,Missing Values,% of Total Values
area_assesed,object,5,0,0.0
district_id,object,31,0,0.0
land_surface_condition,object,3,0,0.0
foundation_type,object,5,0,0.0
roof_type,object,3,0,0.0
ground_floor_type,object,5,0,0.0
other_floor_type,object,4,0,0.0
position,object,4,0,0.0
plan_configuration,object,10,0,0.0
condition_post_eq,object,8,0,0.0


In [19]:
# Groupby and aggregate with your dictionary:
display(data.groupby(['position']).agg({'damage_grade': ['mean']}))
display(data.groupby(['land_surface_condition']).agg({'damage_grade': ['mean']}))
display(data.groupby(['condition_post_eq']).agg({'damage_grade': ['mean']}))

,damage_grade
,mean
position,
Attached-1 side,3.756797
Attached-2 side,3.467851
Attached-3 side,3.750086
Not attached,3.547202


,damage_grade
,mean
land_surface_condition,
Flat,3.566867
Moderate slope,3.624321
Steep slope,3.717455


,damage_grade
,mean
condition_post_eq,
Covered by landslide,5.000000
Damaged-Not used,3.617549
Damaged-Repaired and used,2.599166
Damaged-Rubble Clear-New building built,5.000000
Damaged-Rubble clear,5.000000
Damaged-Rubble unclear,5.000000
Damaged-Used in risk,3.098836
Not damaged,1.001389


In [20]:
# Weight of Evidence encoding
woe_count = 0

for col in combined_set:
    if combined_set[col].dtype == 'object':
        df = data.groupby([col]).agg({'damage_grade': ['mean']})
        df = df['damage_grade']['mean'].sort_values()
        key = list(df.index)
        val = list(df)
        #val = np.arange(len(df))
        combined_set[col].replace({k: round(v,2) for k,v in zip(key, val)}, inplace=True)
        # Keep track of how many columns were label encoded
        woe_count += 1

print('%d columns were woe encoded.' % woe_count)

11 columns were woe encoded.


In [21]:
combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1052936 entries, 0 to 1052935
Data columns (total 51 columns):
area_assesed                              1052936 non-null float64
district_id                               1052936 non-null float64
has_geotechnical_risk                     1052936 non-null float64
has_geotechnical_risk_fault_crack         1052936 non-null int64
has_geotechnical_risk_flood               1052936 non-null int64
has_geotechnical_risk_land_settlement     1052936 non-null int64
has_geotechnical_risk_landslide           1052936 non-null int64
has_geotechnical_risk_liquefaction        1052936 non-null int64
has_geotechnical_risk_other               1052936 non-null int64
has_geotechnical_risk_rock_fall           1052936 non-null int64
has_repair_started                        1052936 non-null float64
count_floors_pre_eq                       1052936 non-null int64
count_floors_post_eq                      1052936 non-null int64
age_building                      

In [22]:
# Create a label encoder object
# le = LabelEncoder()
# le_count = 0

# for col in combined_set:
#     if combined_set[col].dtype == 'object' or col == 'damage_grade':
#         # Train on training data
#         le.fit(combined_set[col])
#         # Transform both training and test data
#         combined_set[col] = le.transform(combined_set[col])
            
#         # Keep track of how many columns were label encoded
#         le_count += 1

# print('%d columns were label encoded.' % le_count)

In [23]:
train_y = data['damage_grade']

In [24]:
train_X = combined_set[:data.shape[0]] # Up to the last initial training set row
test_X = combined_set[data.shape[0]:] # Past the last initial training set row

print('Training Feature Size: ', train_X.shape)
print('Training Target Size: ', train_y.shape)
print('Testing Feature Size: ', test_X.shape)

Training Feature Size:  (631761, 51)
Training Target Size:  (631761,)
Testing Feature Size:  (421175, 51)


# Baseline

## Random Forest

In [36]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, make_scorer

In [33]:
#%timeit
rf = RandomForestClassifier(random_state=0,n_estimators = 250,max_features= 'sqrt', oob_score=True, n_jobs= -1)
rf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=-1,
            oob_score=True, random_state=0, verbose=0, warm_start=False)

In [43]:
rf.oob_score_

0.7430214907219661

In [44]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
print(rf.get_params())

Parameters currently in use:

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 250, 'n_jobs': -1, 'oob_score': True, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [46]:
pred = rf.predict(test_X)

In [47]:
submission = pd.DataFrame({ 'building_id': test.building_id,
                            'damage_grade': pred })

submission['damage_grade'] = submission['damage_grade'].astype(str)
submission['damage_grade'] = "Grade " + submission.loc[:,'damage_grade']
submission.to_csv('rf_with_woe_250trees_maxfeatureSQRT.csv', index=False)

In [48]:
submission.head()

,building_id,damage_grade
0,a3380c4f75,Grade 3
1,a338a4e653,Grade 5
2,a338a4e6b7,Grade 5
3,a33a6eaa3a,Grade 3
4,a33b073ff6,Grade 5


## XGBoost

### Initial Model Setup and Grid Search

In [30]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from datetime import datetime

In [31]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [32]:
def my_custom_loss_func(ground_truth, predictions):
    return(f1_score(y_true=ground_truth,y_pred=predictions,average='weighted'))
score = make_scorer(my_custom_loss_func, greater_is_better=True)

In [35]:
#cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}

st = timer()
print(st)
cv_params = {'max_depth': [3], 'min_child_weight': [1]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 150, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'multi:softprob'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, scoring = score, cv = 5, n_jobs = -1)

2018-07-18 16:15:31.247582


In [ ]:
optimized_GBM.fit(train_X, train_y)
print(timer())
print(timer()-st)

---

In [ ]:
gbm = xgb.XGBClassifier(max_depth=8,
                    learning_rate = 0.01,
                    n_estimators=100,
                    subsample=0.9,
                    colsample_bytree=0.45,
                    objective="multi:softprob",
                    silent = False,
                    min_child_weight=1,                       
                    nthread=-1)

In [ ]:
gbm.fit(train_X, train_y, eval_metric="mlogloss",
        eval_set = [(train_X, train_y)], verbose=10)                        

In [ ]:
xgb_pred = gbm.predict(test_X)

In [ ]:
submission = pd.DataFrame({ 'building_id': test.building_id,
                            'damage_grade': xgb_pred })

submission['damage_grade'] = submission['damage_grade'].astype(str)
submission['damage_grade'] = "Grade " + submission.loc[:,'damage_grade']
submission.to_csv('xgboost_woe_100estimators.csv', index=False)

---

In [ ]:
gbm = xgb.XGBClassifier(max_depth=6, n_estimators=150, learning_rate=0.5, objective= "multi:softprob", silent=False)
gbm.fit(train_X, train_y, eval_metric='mlogloss')

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

fit_params = {'eval_metric': 'mlogloss'}

In [ ]:
xgbc = xgb.XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softprob', silent=True, nthread=1)

In [ ]:
folds = 5
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgbc, param_distributions=params, fit_params=fit_params, n_iter=param_comb, scoring='f1_samples', n_jobs=4, cv=skf.split(train_X,train_y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(train_X, train_y)
timer(start_time) # timing ends here for "start_time" variable

In [ ]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)

In [ ]:
pred = gbm.predict(test_data)

In [ ]:
from sklearn import svm

clf = svm.LinearSVC()
clf.fit(train_X,train_y)
pred = clf.predict(test_data)

In [ ]:
pred = pred + 1

In [ ]:
submission = pd.DataFrame({ 'building_id': test.building_id,
                            'damage_grade': pred })

submission['damage_grade'] = submission['damage_grade'].astype(str)
submission['damage_grade'] = "Grade " + submission.loc[:,'damage_grade'] 

In [ ]:
submission.to_csv('Dataset/submission_8.csv', index=False)